<a href="https://colab.research.google.com/github/simon108018/Machine-Learning/blob/master/InceptionV3(tflite).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers, initializers
from tensorflow.keras.utils import plot_model
from IPython.display import Image
import tensorflow_datasets as tfds
import logging
logging.getLogger("tensorflow").setLevel(logging.DEBUG)
assert float(tf.__version__[:3]) >= 2.3

#tfds.list_builders()

In [ ]:
# 讀取model
model_dir = '/content/drive/My Drive/Colab Notebooks/cifar10/models'
model = tf.keras.models.load_model(model_dir + "/Best-model-InceptionV3-L2.h5")

In [ ]:
# 資料處理
test_data, info = tfds.load("cifar10", split="test", with_info=True)

In [ ]:
# 定義圖像縮放
def parse_aug_fn(dataset):
    def zoom(x, sh = 299, sw = 299):
        x = tf.image.resize(x, (sh, sw))
        return x
    
    # 影像標準化
    x = tf.cast(dataset['image'], tf.float32) / 255.
    # 影像放大到224*224
    x = zoom(x)
    y = tf.one_hot(dataset['label'], 10)
    y = y
    
    return x, y

In [ ]:
# 資料處理
batch_size = 128
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_num = int(info.splits['train'].num_examples / 10) * 9
test_data = test_data.map(map_func=parse_aug_fn, num_parallel_calls=AUTOTUNE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)
test_data_batch = test_data.batch(batch_size)

In [ ]:
# 評估精準度(用batch加速計算)
model.evaluate(test_data_batch)

79/79 [==============================] - 89s 1s/step - loss: 0.5511 - categorical_accuracy: 0.8439


[0.5510784983634949, 0.8439000248908997]

In [ ]:
# 簡單轉換成tf.lite形式
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_float_model = converter.convert()

# Show model size in KBs.
float_model_size = len(tflite_float_model) / 1024
print('Float model size = %dKBs.' % float_model_size)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: /tmp/tmpp7mraqia/assets


INFO:tensorflow:Assets written to: /tmp/tmpp7mraqia/assets


Float model size = 93327KBs.


In [ ]:
# 簡單Quantize model
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

# Show model size in KBs.
quantized_model_size = len(tflite_quantized_model) / 1024
print('Quantized model size = %dKBs,' % quantized_model_size)
print('which is about %d%% of the float model size.'\
      % (quantized_model_size * 100 / float_model_size))

INFO:tensorflow:Assets written to: /tmp/tmp8nedabq_/assets


INFO:tensorflow:Assets written to: /tmp/tmp8nedabq_/assets


Quantized model size = 23437KBs,
which is about 25% of the float model size.


In [ ]:
# 設定測試資料量
n = int(input("請輸入test數量\n"))
test_data_np = list(tfds.as_numpy(test_data.take(n)))

## 會使用list是因為tfds.as_numpy產生出了的是generator
## 在下面evaluate_tflite_model的function中，
## 如果要重複呼叫函數數次，轉成list才不會error，
## 因為generator執行過後的值會不見(節省記憶體)。

請輸入test數量
1000


In [ ]:
# tflite的評估函數

def evaluate_tflite_model(tflite_model):
  # Initialize TFLite interpreter using the model.
  interpreter = tf.lite.Interpreter(model_content=tflite_model)
  interpreter.allocate_tensors()
  input_tensor_index = interpreter.get_input_details()[0]["index"]
  output = interpreter.tensor(interpreter.get_output_details()[0]["index"])
 
  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  i = 0
  accurate_count = 0
  test_labels = []
  for data in test_data_np:
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    # test_image = np.expand_dims(np.array(data[0]), axis=0).astype(np.float32)
    test_image = np.expand_dims(data[0], axis=0).astype(np.float32)
    interpreter.set_tensor(input_tensor_index, test_image)
 
    # Run inference.
    interpreter.invoke()
 
    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)
    
    test_labels.append(np.argmax(data[1]))
    # print("digit:",prediction_digits[i],"labels:",labels[i])
 
    
    i+=1
    if i%100 == 0:
      j=i*100.0/n
      print('目前進度為{j}%。'.format(j=j))
      # for index in range(len(prediction_digits)):
      #   if prediction_digits[index] == test_labels[index]:
      #     accurate_count += 1
      # accuracy = accurate_count * 1.0 / len(prediction_digits)
      # print(accuracy)
    
    
 
  # Compare prediction results with ground truth labels to calculate accuracy.
  
  for index in range(len(prediction_digits)):
    if prediction_digits[index] == test_labels[index]:
      accurate_count += 1
  accuracy = accurate_count * 1.0 / len(prediction_digits)
 
  return accuracy

In [ ]:
print('tflite_float_model:')
float_accuracy = evaluate_tflite_model(tflite_float_model)
print('Float model accuracy = %.4f' % float_accuracy)
print('tflite_quantized_model:')
quantized_accuracy = evaluate_tflite_model(tflite_quantized_model)
print('Quantized model accuracy = %.4f' % quantized_accuracy)

tflite_float_model:
目前進度為10.0%。
目前進度為20.0%。
目前進度為30.0%。
目前進度為40.0%。
目前進度為50.0%。
目前進度為60.0%。
目前進度為70.0%。
目前進度為80.0%。
目前進度為90.0%。
目前進度為100.0%。
Float model accuracy = 0.8490
tflite_quantized_model:
目前進度為10.0%。
目前進度為20.0%。
目前進度為30.0%。
目前進度為40.0%。
目前進度為50.0%。
目前進度為60.0%。
目前進度為70.0%。
目前進度為80.0%。
目前進度為90.0%。
目前進度為100.0%。
Quantized model accuracy = 0.7920
